In [2]:
import graspy
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def gen_data(n, eps):
    pi = [0.4, 0.6]
    sizes = [int(pi[0]*n), int(pi[1]*n)]
    
    probsA = np.array([
        [0.5, 0.2],
        [0.2, 0.5]])
    
    probsB = np.array([
        [0.5 + eps, 0.2],
        [0.2, 0.5 + eps]])
    A1 = sbm(n=sizes, p=probsA)
    A2 = sbm(n=sizes, p=probsB)
    return A1, A2

def estimated_power(n, eps, iters):
    A1, A2 = gen_data(n, eps)
    npt = graspy.inference.NonparametricTest()
    npt.fit(A1, A2)
    sorted_ = np.sort(npt.null_distribution_)
    rej_ind = int(np.ceil(((1 - 0.05)*200))) #200 is n_bootstraps, 0.05 is alpha
    critical_value = sorted_[rej_ind]
    
    rejections = 0
    for i in range(iters):
        A, B = gen_data(n, eps)
        npt_2 = graspy.inference.NonparametricTest(n_bootstraps=1)
        npt_2.fit(A,B)
        U = npt_2.sample_T_statistic_
        if U > critical_value:
            rejections += 1
    return rejections/iters

def monte_carlo(ns, eps, iters = 1000):
    powers = np.zeros(shape = (len(ns),len(eps)))
    for i in tqdm(range(len(ns))):
        for j in tqdm(range(len(eps))):
            powers[i,j] = np.array(estimated_power(ns[i], eps[j], iters))
    return powers